In [1]:
from updated_qsp_dataset import UpdatedWebQSPDataset
from raw_qsp_dataset import RawWebQSPDataset
from unittest.mock import patch

/home/zaristei/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
old_mock, new_mock = None, None
old_dataset = RawWebQSPDataset(root='old_dataset', force_reload=True, limit=2, with_process=True, with_pcst=True)
new_dataset = UpdatedWebQSPDataset(root='updated_dataset',force_reload=True, limit=2, whole_graph_retrieval=False)

Processing...


Encoding questions...
Encoding graphs...


100%|██████████| 2/2 [00:07<00:00,  3.63s/it]
Done!
Processing...


Encoding graph...
Encoding questions...
Retrieving subgraphs...


100%|██████████| 2/2 [00:00<00:00,  3.69it/s]
Done!


In [6]:
old_dataset[0]

Data(x=[12, 1024], edge_index=[2, 12], edge_attr=[12, 1024], question='Question: what is the name of justin bieber brother
Answer: ', label='jaxon bieber', desc='node_id,node_attr
15,justin bieber
151,pattie mallette
286,english language
294,jaxon bieber
346,yves bole
356,jeremy bieber
452,jazmyn bieber
545,m.0wfn4pm
551,m.0gxnnwp
933,m.0gxnnwc
1032,this is justin bieber
1359,m.0129jzth

src,edge_attr,dst
346,people.person.languages,286
1032,film.film.language,286
346,influence.influence_node.influenced_by,15
151,people.person.children,15
294,people.person.parents,356
545,people.sibling_relationship.sibling,151
933,people.sibling_relationship.sibling,452
1359,people.sibling_relationship.sibling,346
551,people.sibling_relationship.sibling,294
15,people.person.sibling_s,933
933,people.sibling_relationship.sibling,15
551,people.sibling_relationship.sibling,15
', num_nodes=12)

In [7]:
new_dataset[0]

Data(x=[15, 1024], edge_index=[2, 20], edge_attr=[20, 1024], question='Question: what is the name of justin bieber brother
Answer: ', label='jaxon bieber', desc='index,node_id,node_attr
54,54,m.0gxnp5m
307,307,m.0129jzth
391,391,pattie mallette
650,650,english language
837,837,justin bieber
934,934,this is justin bieber
1022,1022,m.0gxnp5x
1123,1123,jeremy bieber
1149,1149,m.0gxnnwp
1847,1847,m.0gxnnwc
2294,2294,yves bole
2702,2702,m.0gxnp5d
2719,2719,jazmyn bieber
2880,2880,m.0wfn4pm
2882,2882,jaxon bieber

src,edge_attr,dst
2294,people.person.languages,650
934,film.film.language,650
2294,influence.influence_node.influenced_by,837
391,people.person.children,837
2882,people.person.parents,1123
2880,people.sibling_relationship.sibling,391
2882,people.person.sibling_s,1149
837,base.popstra.celebrity.hangout,1022
1847,people.sibling_relationship.sibling,2719
307,people.sibling_relationship.sibling,2294
391,people.person.sibling_s,2880
1149,people.sibling_relationship.sibling,2882
2294,peo

In [8]:
old_dataset.raw_graphs[0]

Data(x=[1709, 1024], edge_index=[2, 9088], edge_attr=[9088, 1024], num_nodes=1709)

In [9]:
new_dataset.raw_graphs[0]

Data(x=[1709, 1024], edge_index=[2, 9088], edge_attr=[9088, 1024], num_nodes=1709, pid=[1709], e_pid=[9088], node_idx=[1709], edge_idx=[9088])

In [10]:
import networkx as nx
from torch_geometric.data import Data
import torch

In [25]:
def results_are_close_enough(ground_truth: Data, new_method: Data, thresh=.9):
    def _sorted_tensors_are_close(tensor1, tensor2):
        vals = torch.isclose(tensor1.sort(dim=0)[0], tensor2.sort(dim=0)[0]).float().mean(axis=1)
        return torch.all(vals > thresh)
    def _graphs_are_same(tensor1, tensor2):
        return nx.weisfeiler_lehman_graph_hash(nx.Graph(tensor1.T)) == nx.weisfeiler_lehman_graph_hash(nx.Graph(tensor2.T))
    val = _sorted_tensors_are_close(ground_truth.x, new_method.x)
    val &= _sorted_tensors_are_close(ground_truth.edge_attr, new_method.edge_attr)
    val &= _graphs_are_same(ground_truth.edge_index, new_method.edge_index)
    return val

In [28]:
results_are_close_enough(old_dataset.raw_graphs[0], new_dataset.raw_graphs[0])

tensor(False)

In [ ]:
old_dataset.questions[0]

In [ ]:
new_dataset.questions[0]

In [ ]:
torch.sort(new_dataset.raw_graphs[0].edge_attr, 0)[0].unique(dim=0).size(0)

In [ ]:
torch.sort(old_dataset.raw_graphs[0].edge_attr, 0)[0].unique(dim=0).size()